In [14]:
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent, AgentType
import pandas as pd

import os

In [2]:
load_dotenv()

True

In [3]:
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
# Step 1: Set up database connection with SQLAlchemy
def initialize_db_connection():
    """Initialize SQLAlchemy engine and session for MySQL database."""
    try:
        engine = create_engine(
            f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}",
            echo=True, # Verbose logging for debugging
            pool_pre_ping=True # Ensure active connections
        )
        Session = sessionmaker(bind=engine)
        session = Session()
        print("Database connection established!")
        return engine, session
    except Exception as e:
        print(f"Failed to connect to database: {e}")
        return None, None

# Step 2: Configure LangChain SQLDatabase
def get_sql_database(engine):
    """Wrap SQLAlchemy engine in LangChain's SQLDatabase."""
    try:
        db = SQLDatabase(engine)
        print("LangChain SQLDatabase initialized!")
        return db
    except Exception as e:
        print(f"Failed to initialize SQLDatabase: {e}")
        return None

# Step 3: Initialize OpenAI LLM
def initialize_llm():
    """Set up OpenAI LLM with API key"""
    try:
        llm = ChatOpenAI(
            model="gpt-4o-mini", # Cost-effective model: adjust as needed
            api_key=OPENAI_API_KEY,
            temperature=0
        )
        print("OpenAI LLM initialized!")
        return llm
    except Exception as e:
        print(f"Failed to initialize LLM: {e}")
        return None

# Step 4: Create SQL Agent
def create_agent(db, llm):
    """Build SQL agent with LangChain toolkit."""
    try:
        toolkit = SQLDatabaseToolkit(db=db, llm=llm)
        agent = create_sql_agent(
            llm=llm,
            toolkit=toolkit,
            agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # React-based reasoning
            verbose=True, # Show intermediate steps
        )
        print("SQL Agent created successfully!")
        return agent
    except Exception as e:
        print(f"Failed to create agent: {e}")
        return None

# Step 5: Process Query and Format Output
def run_query(agent, query):
    """Execute a natural language query and return formatted response."""
    try:
        response = agent.run(query)
        # Format raw SQL output into natural language
        if "No results found" in response:
            return f"Sorry, I couldn't find any data for : '{query}'."
        else:
            return f"Here's what I found for '{query}':\n{response}"
    except Exception as e:
        return f"Error processing query '{query}': {e}"
        
        

In [5]:
# Set up
engine, session = initialize_db_connection()
if engine and session:
    db = get_sql_database(engine)
    if db:
        llm = initialize_llm()
        if llm:
            agent = create_agent(db, llm)

Database connection established!
2025-03-27 13:13:54,400 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-03-27 13:13:54,400 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-27 13:13:54,403 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-03-27 13:13:54,404 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-27 13:13:54,405 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-03-27 13:13:54,406 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-27 13:13:54,407 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-27 13:13:54,408 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `finquery`
2025-03-27 13:13:54,408 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-27 13:13:54,426 INFO sqlalchemy.engine.Engine ROLLBACK
2025-03-27 13:13:54,428 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-27 13:13:54,428 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `finquery`
2025-03-27 13:13:54,429 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-27 13:13:54,441

In [6]:
query = "What is the largest amount in the financial_transactions table?"
result = run_query(agent, query)
print(result)

/var/folders/fl/jdrswfjx50v2304lpb3s4wxr0000gn/T/ipykernel_96155/2214542209.py:65: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  accounting_transactions, financial_transactionsI need to check the schema of the `financial_transactions` table to identify the relevant columns for my query.  
Action: sql_db_schema  
Action Input: "financial_transactions"  2025-03-27 13:14:00,712 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-27 13:14:00,713 INFO sqlalchemy.engine.Engine SELECT financial_transactions.transaction_id, financial_transactions.date, financial_transactions.customer_id, financial_transactions.amount, financial_transactions.type, financial_transactions.description 
FROM financial_transactions 
 LIMIT %(param_1)s
2025-03-27 13:14:00,714 INFO sqlalchemy.engine.Engine [generated in 0.00136s] {'param_1': 3}
2025-03-27 13:14:00,716 INFO sqlalchemy.engine.Engine ROLLBACK

CREATE TABLE financial_transactions (
	transaction_id BIGINT NOT NULL, 
	date DATE, 
	customer_id BIGINT NOT NULL, 
	amount DECIMAL(15, 2), 
	type

In [7]:
query = "What is the smallest amount in the financial_transactions table?"
result = run_query(agent, query)
print(result)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  accounting_transactions, financial_transactionsI will now check the schema of the financial_transactions table to understand its structure and identify the relevant columns for my query.  
Action: sql_db_schema  
Action Input: "financial_transactions"  2025-03-27 13:14:22,480 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-27 13:14:22,480 INFO sqlalchemy.engine.Engine SELECT financial_transactions.transaction_id, financial_transactions.date, financial_transactions.customer_id, financial_transactions.amount, financial_transactions.type, financial_transactions.description 
FROM financial_transactions 
 LIMIT %(param_1)s
2025-03-27 13:14:22,481 INFO sqlalchemy.engine.Engine [cached since 21.77s ago] {'param_1': 3}
2025-03-27 13:14:22,483 INFO sqlalchemy.engine.Engine ROLLBACK

CREATE TABLE financial_transactions (
	transaction_id BIGINT NOT NULL, 
	date DATE, 
	customer_id BIGINT NOT NULL, 


In [8]:
query = "What is the average amount in the financial_transactions table?"
result = run_query(agent, query)
print(result)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  accounting_transactions, financial_transactionsI need to check the schema of the `financial_transactions` table to identify the relevant columns for calculating the average amount.  
Action: sql_db_schema  
Action Input: "financial_transactions"  2025-03-27 13:14:32,469 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-27 13:14:32,469 INFO sqlalchemy.engine.Engine SELECT financial_transactions.transaction_id, financial_transactions.date, financial_transactions.customer_id, financial_transactions.amount, financial_transactions.type, financial_transactions.description 
FROM financial_transactions 
 LIMIT %(param_1)s
2025-03-27 13:14:32,469 INFO sqlalchemy.engine.Engine [cached since 31.76s ago] {'param_1': 3}
2025-03-27 13:14:32,471 INFO sqlalchemy.engine.Engine ROLLBACK

CREATE TABLE financial_transactions (
	transaction_id BIGINT NOT NULL, 
	date DATE, 
	customer_id BIGINT NOT NULL, 
	amoun

In [9]:
query = "How much did customer 926 spend?"
result = run_query(agent, query)
print(result)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  accounting_transactions, financial_transactionsI need to check the schema of the relevant tables to find out where customer spending information is stored. Since "accounting_transactions" and "financial_transactions" are the only tables available, I will check their schemas to see which one contains customer spending data. 

Action: sql_db_schema  
Action Input: "accounting_transactions, financial_transactions"  2025-03-27 13:14:51,893 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-27 13:14:51,894 INFO sqlalchemy.engine.Engine SELECT accounting_transactions.`Transaction_ID`, accounting_transactions.`Date`, accounting_transactions.`Account_Number`, accounting_transactions.`Transaction_Type`, accounting_transactions.`Amount`, accounting_transactions.`Currency`, accounting_transactions.`Counterparty`, accounting_transactions.`Category`, accounting_transactions.`Payment_Method`, accounting_t

In [9]:
query = "How much did customer 926 spend?"
result = run_query(agent, query)
print(result)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  accounting_transactions, financial_transactionsI need to check the schema of the relevant tables to find out where customer spending information is stored. Since "accounting_transactions" and "financial_transactions" are the only tables available, I will check their schemas to see which one contains customer spending data. 

Action: sql_db_schema  
Action Input: "accounting_transactions, financial_transactions"  2025-03-27 13:14:51,893 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-27 13:14:51,894 INFO sqlalchemy.engine.Engine SELECT accounting_transactions.`Transaction_ID`, accounting_transactions.`Date`, accounting_transactions.`Account_Number`, accounting_transactions.`Transaction_Type`, accounting_transactions.`Amount`, accounting_transactions.`Currency`, accounting_transactions.`Counterparty`, accounting_transactions.`Category`, accounting_transactions.`Payment_Method`, accounting_t

In [10]:
query = "Get monthly sales data"
result = run_query(agent, query)
print('-------')
print(result)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  accounting_transactions, financial_transactionsI need to check the schema of the relevant tables to find out which one contains the monthly sales data. Since "accounting_transactions" and "financial_transactions" could both potentially have sales data, I will check their schemas. 

Action: sql_db_schema  
Action Input: "accounting_transactions, financial_transactions"  2025-03-27 13:17:45,129 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-27 13:17:45,130 INFO sqlalchemy.engine.Engine SELECT accounting_transactions.`Transaction_ID`, accounting_transactions.`Date`, accounting_transactions.`Account_Number`, accounting_transactions.`Transaction_Type`, accounting_transactions.`Amount`, accounting_transactions.`Currency`, accounting_transactions.`Counterparty`, accounting_transactions.`Category`, accounting_transactions.`Payment_Method`, accounting_transactions.`Risk_Incident`, accounting_tran

In [11]:
query = """
Show me a line chart of the monthly sales data.
Write only the Python code using plotly to create this chart.  Use pandas to load data.
"""
result = run_query(agent, query)
print('-------')
print(result)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  accounting_transactions, financial_transactionsI should check the schema of the tables related to transactions to find the relevant columns for sales data. Since "accounting_transactions" and "financial_transactions" are the only tables, I will query their schemas.  
Action: sql_db_schema  
Action Input: "accounting_transactions, financial_transactions"  2025-03-27 13:19:40,177 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-27 13:19:40,177 INFO sqlalchemy.engine.Engine SELECT accounting_transactions.`Transaction_ID`, accounting_transactions.`Date`, accounting_transactions.`Account_Number`, accounting_transactions.`Transaction_Type`, accounting_transactions.`Amount`, accounting_transactions.`Currency`, accounting_transactions.`Counterparty`, accounting_transactions.`Category`, accounting_transactions.`Payment_Method`, accounting_transactions.`Risk_Incident`, accounting_transactions.`Risk_

In [12]:
query = """
Give me a line chart of the monthly sales data.
Write only the sql code to create this chart.
"""
result = run_query(agent, query)
print('-------')
print(result)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  accounting_transactions, financial_transactionsI need to check the schema of the relevant tables to find the columns that contain sales data. I will start with the `accounting_transactions` table.  
Action: sql_db_schema  
Action Input: "accounting_transactions"  2025-03-27 13:24:21,546 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-27 13:24:21,548 INFO sqlalchemy.engine.Engine SELECT accounting_transactions.`Transaction_ID`, accounting_transactions.`Date`, accounting_transactions.`Account_Number`, accounting_transactions.`Transaction_Type`, accounting_transactions.`Amount`, accounting_transactions.`Currency`, accounting_transactions.`Counterparty`, accounting_transactions.`Category`, accounting_transactions.`Payment_Method`, accounting_transactions.`Risk_Incident`, accounting_transactions.`Risk_Type`, accounting_transactions.`Incident_Severity`, accounting_transactions.`Error_Code`, acc

In [17]:
sql_query = """SELECT DATE_FORMAT(date, '%Y-%m') AS month, SUM(amount) AS total_sales 
FROM financial_transactions 
GROUP BY month 
ORDER BY month 
LIMIT 10;"""

In [18]:
with engine.connect() as connection:
    df = pd.read_sql(sql=text(sql_query), con=connection)

2025-03-27 13:27:29,768 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-27 13:27:29,770 INFO sqlalchemy.engine.Engine SELECT DATE_FORMAT(date, '%Y-%m') AS month, SUM(amount) AS total_sales 
FROM financial_transactions 
GROUP BY month 
ORDER BY month 
LIMIT 10;
2025-03-27 13:27:29,771 INFO sqlalchemy.engine.Engine [generated in 0.00300s] {}
2025-03-27 13:27:29,867 INFO sqlalchemy.engine.Engine ROLLBACK


In [19]:
df

,month,total_sales
0,2018-03,1411092.04
1,2018-04,8165918.47
2,2018-05,8757920.69
3,2018-06,8229321.90
4,2018-07,8717899.94
5,2018-08,8658032.26
6,2018-09,8395163.23
7,2018-10,8296004.70
8,2018-11,8226845.34
9,2018-12,8355343.64


In [21]:
df.head().to_markdown()

ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.